In [1]:
import os
import pandas as pd
import glob
import numpy as np

In [22]:
#all files in the folde rcontaining all the stock sheets are saved in our neighbour folder into databases then merging all the dfs together into 1
#the dataframes will stack on top of each other so that the number of columns remains when merging databases to each individual database

#find the neighbour folder and saving all the files in said folder into a List called fileNames
folder_path = './PythonData'
fileNames = glob.glob(os.path.join(folder_path, "*.csv"))


#creating an empty array to add all the databases to
dfList=[]

#convering each csv file to a database and then appending it into the dfList[] 
for i in range (0, len(fileNames)):
    if os.path.isfile(fileNames[i]):
        df = pd.read_csv(f'{fileNames[i]}') 
        ticker = fileNames[i].replace('.csv', '')#getting the ticker value from the array of file names
        df['Ticker']=ticker#adding a ticker column with the rigasxdcv b,ht 'ticker' value for the file in the iteration
    
        dfList.append(df)

#concatenating all the databases into one
merged_df=pd.concat(dfList, ignore_index = True)


In [15]:
#creating empty arrays to store the max, min prices and returns and other calculations for each stock sheet/ticker
max_prices=[]
max_returns=[]
min_prices=[]
min_returns=[]
price_stds=[]
return_stds=[]
date_of_max_prices=[]
percent_returns=[]
log_returns=[]

#creating a function to calculate percentage returns
def PercentReturnCalculator (localReturn):
    percentageReturn=localReturn*100
    percentageReturnString=(str(percentageReturn)+"%")
    return percentageReturnString

#creating a function to calculate log returns
def LogReturnCalculator (localReturn):
    if (localReturn>-1):
        logReturn=(np.log1p(localReturn))
    else:
        logReturn = "the return value has to be greater than -1"
    return logReturn

for e in range(0, len(fileNames)):#iterating through each file/stock sheet
    returns=[]#creating an empty array to store all the return values in the file that is being iterated through
    '''
        creating a seperate dataframe out of the merged dataframe for each ticker/stock sheet by finding all the rows 
        that have the same value under the 'Ticker' column
    '''
    cutted_df = merged_df[merged_df['Ticker']==fileNames[e].replace('.csv', '')]
    #appending the maximum and minimum prices in the cutted_df to the max_prices and min_prices arrays
    maxValue=float(cutted_df['Adj Close'].max())
    max_prices.append(maxValue)
    min_prices.append(float(cutted_df['Adj Close'].min()))

    singular_percent_returns = []
    singular_log_returns = []
    for a in range(0, len(cutted_df)-1):#iterating to calculate each return value in the cutted_df      
        singular_return=(merged_df.iloc[a+1]['Adj Close']-merged_df.iloc[a]['Adj Close'])
       
        #calculating the percentage return for each return value and appending it into a temporary list
        singular_percent_return = PercentReturnCalculator(singular_return)
        singular_percent_returns.append(singular_percent_return) 
        
        #doing the same thing for log returns as percentage returns above
        singular_log_return = LogReturnCalculator(singular_return)
        singular_log_returns.append(singular_log_return)

        returns.append(singular_return)#appending each value to the empty array initialized at the start of the initial for loop   
    
    percent_returns.append(singular_percent_returns)#append the temporary list of percentage returns as a nested list inside [percent_returns]
    log_returns.append(singular_log_returns)
    
    #appending the maximum and minimum return values and std values into their respective arrays
    max_returns.append(float(max(returns)))
    min_returns.append(float(min(returns)))
    price_stds.append(cutted_df['Adj Close'].std())
    return_stds.append(np.std(returns))
    
    #getting the value under the 'Date' column in the same row where 'Adj Close'==maxValue
    date_of_max_prices.append(cutted_df.loc[cutted_df['Adj Close']==maxValue, 'Date'].iloc[0])

In [21]:
#creating a new database to store all calculations where the arrays are columns
values_db={
    'maximum prices': max_prices,
    'maximum returns': max_returns,
    'minimum prices': min_prices,
    'minimum returns': min_returns,
    'standard deviation of all prices': price_stds,
    'standard deviation of all returns': return_stds,
    'date_of_max_prices': date_of_max_prices
}

#creating lists to store series objects of percentage and log returns from the elements in the nested lists
percent_series_list=[]
log_series_list=[]

#iterating through each stock sheet to turn each element (all the returns for one stock sheet) in the nested lists into a series 
#appending those series' into the lists created above
for i in range(0, len(fileNames)):
    percent_series_list.append(pd.Series(percent_returns[i], name=f'{fileNames[i]}'))
    log_series_list.append(pd.Series(log_returns[i], name=f'{fileNames[i]}'))

#concatenating all series' into dataframes, the series are first stored into lists to allow for the columns in the dataframes to be different lengths
percent_df=pd.concat(percent_series_list, axis=1)
log_df=pd.concat(log_series_list, axis=1)

Stock_Sheet_Analysis=pd.DataFrame(values_db, index =fileNames)

#creating an excel file from the database created above
with pd.ExcelWriter('Stock Sheet Data Calculations.xlsx') as writer:

        Stock_Sheet_Analysis.to_excel(writer, sheet_name='Sheet 1')
        percent_df.to_excel(writer, sheet_name='Percentage Returns')
        log_df.to_excel(writer, sheet_name='Log Returns')
